# Hilbert–Schmidt Independence Criterion (HSIC): A Clear Numerical Example

This section provides a **step-by-step numerical explanation** of **HSIC** using the **same nonlinear example** as before. HSIC is a **kernel-based, nonparametric dependence measure** that is widely used for **feature selection**, especially in **time series and high-dimensional settings**.

Unlike Mutual Information (MI), HSIC:

- Does **not** estimate probability densities
- Does **not** require discretization or kNN
- Measures dependence via **kernels and geometry in RKHS**

---

## Example Setup

We observe one feature \(X\) and one target \(Y\) over four time points:

| i   | \(X_i\) | \(Y_i\) |
| --- | ------- | ------- |
| 1   | 1       | 1       |
| 2   | 2       | 4       |
| 3   | 3       | 9       |
| 4   | 4       | 16      |

This is again a **nonlinear relationship**:  
\(Y = X^2\)

---

## Step 1 — Choose Kernels

HSIC requires a kernel for \(X\) and a kernel for \(Y\).

For simplicity, we use the **linear kernel**:

$$
k(x_i, x_j) = x_i x_j
$$

$$
l(y_i, y_j) = y_i y_j
$$

> Note: In practice, **Gaussian (RBF) kernels** are more common, but linear kernels make the computations transparent.

---

## Step 2 — Compute Kernel Matrices

### Kernel Matrix for \(X\)

$$
K_{ij} = X_i X_j
$$

| \(K\) | 1   | 2   | 3   | 4   |
| ----- | --- | --- | --- | --- |
| 1     | 1   | 2   | 3   | 4   |
| 2     | 2   | 4   | 6   | 8   |
| 3     | 3   | 6   | 9   | 12  |
| 4     | 4   | 8   | 12  | 16  |

---

### Kernel Matrix for \(Y\)

$$
L_{ij} = Y_i Y_j
$$

| \(L\) | 1   | 2   | 3   | 4   |
| ----- | --- | --- | --- | --- |
| 1     | 1   | 4   | 9   | 16  |
| 2     | 4   | 16  | 36  | 64  |
| 3     | 9   | 36  | 81  | 144 |
| 4     | 16  | 64  | 144 | 256 |

---

## Step 3 — Center the Kernel Matrices

HSIC operates on **centered kernel matrices**.

Define the centering matrix:

$$
H = I - \frac{1}{n}\mathbf{1}\mathbf{1}^\top
$$

For \(n = 4\):

$$
H =
\begin{bmatrix}
0.75 & -0.25 & -0.25 & -0.25 \\
-0.25 & 0.75 & -0.25 & -0.25 \\
-0.25 & -0.25 & 0.75 & -0.25 \\
-0.25 & -0.25 & -0.25 & 0.75
\end{bmatrix}
$$

Centered kernels:

$$
\tilde{K} = HKH
$$

$$
\tilde{L} = HLH
$$

---

### Centered Kernel Matrix for \(X\)

| \(\tilde{K}\) | 1    | 2    | 3    | 4    |
| ------------- | ---- | ---- | ---- | ---- |
| 1             | 3.5  | 1.5  | -0.5 | -4.5 |
| 2             | 1.5  | 0.5  | -0.5 | -1.5 |
| 3             | -0.5 | -0.5 | 0.5  | 0.5  |
| 4             | -4.5 | -1.5 | 0.5  | 5.5  |

---

### Centered Kernel Matrix for \(Y\)

| \(\tilde{L}\) | 1     | 2     | 3     | 4     |
| ------------- | ----- | ----- | ----- | ----- |
| 1             | 73.5  | 25.5  | -18.5 | -80.5 |
| 2             | 25.5  | 7.5   | -6.5  | -26.5 |
| 3             | -18.5 | -6.5  | 7.5   | 17.5  |
| 4             | -80.5 | -26.5 | 17.5  | 89.5  |

---

## Step 4 — Compute HSIC

The empirical HSIC is defined as:

$$
\mathrm{HSIC}(X,Y)
=
\frac{1}{(n-1)^2}
\mathrm{trace}(\tilde{K}\tilde{L})
$$

---

### Matrix Product and Trace

Compute:

$$
\mathrm{trace}(\tilde{K}\tilde{L})
=
\sum_{i,j} \tilde{K}_{ij}\tilde{L}_{ij}
$$

For this example:

$$
\sum_{i,j} \tilde{K}_{ij}\tilde{L}_{ij} = 2260
$$

With \(n = 4\):

$$
\mathrm{HSIC}(X,Y) = \frac{2260}{9} \approx 251.1
$$

---

## Interpretation

- HSIC \(> 0\) indicates dependence
- HSIC \(= 0\) **if and only if** \(X\) and \(Y\) are independent (with characteristic kernels)
- The large value reflects **strong nonlinear dependence**

---

## Why HSIC is Powerful for Time Series Feature Selection

- Fully **nonparametric**
- Captures **linear and nonlinear** dependencies
- Avoids density estimation (unlike MI)
- Works naturally with **lagged variables**
- Stable in **high dimensions**

---

## Comparison to MI and dCor

| Method               | Density Estimation | Captures Nonlinearity | Popular in TS FS |
| -------------------- | ------------------ | --------------------- | ---------------- |
| Mutual Information   | Yes (kNN / KDE)    | Yes                   | Very high        |
| Distance Correlation | No                 | Yes                   | High             |
| HSIC                 | No (kernel-based)  | Yes                   | Very high        |

---

## Key Takeaway

> HSIC measures dependence by comparing **similarity structures** in RKHS using kernels.

This makes it one of the **most robust and widely adopted tools** for:

- Multivariate time series feature selection
- Lag selection
- Nonlinear dependency detection
- Modern ML pipelines


In [1]:
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

In [2]:
def create_lags(x, L):
    """
    Create lagged matrix of a univariate time series.

    Parameters
    ----------
    x : array-like, shape (N,)
        Time series feature
    L : int
        Maximum lag

    Returns
    -------
    X_lag : ndarray, shape (N-L, L)
    y_aligned_index : ndarray
        Indices aligning lagged matrix with target
    """
    x = np.asarray(x)
    N = len(x)

    X_lag = np.column_stack([x[L - j - 1:N - j - 1] for j in range(L)])
    return X_lag


In [3]:
def kernel_distance_columnwise(X_lag, y, gamma_x=1.0, gamma_y=1.0):
    """
    Compute kernel-induced distance matrix between lagged features and target.

    Parameters
    ----------
    X_lag : ndarray, shape (N, L)
        Lagged feature matrix
    y : ndarray, shape (N,)
        Target vector
    gamma_x : float
        Kernel bandwidth for X
    gamma_y : float
        Kernel bandwidth for y

    Returns
    -------
    D : ndarray, shape (N, L)
        Distance matrix
    """
    N, L = X_lag.shape
    y = y.reshape(-1, 1)

    Ky = rbf_kernel(y, y, gamma=gamma_y)
    Ly_diag = np.diag(Ky/np.linalg.norm(Ky, ord='fro'))

    D = np.zeros((N, L))

    for j in range(L):
        xj = X_lag[:, j].reshape(-1, 1)

        Kx = rbf_kernel(xj, xj, gamma=gamma_x)
        Kxy = rbf_kernel(xj, y, gamma=gamma_x)

        kx_norm = Kx/np.linalg.norm(Kx, ord='fro')
        kxy_norm = Kxy/np.linalg.norm(Kxy, ord='fro')

        D[:, j] = np.sqrt(
            np.diag(kx_norm) + Ly_diag - 2 * np.diag(kxy_norm)
        )

    return D


In [4]:
def normalize_columns(D):
    """
    Normalize each column independently to [0,1].

    Parameters
    ----------
    D : ndarray, shape (N, L)

    Returns
    -------
    D_norm : ndarray, shape (N, L)
    """
    D_norm = np.zeros_like(D)

    for j in range(D.shape[1]):
        col = D[:, j]
        min_val = np.min(col)
        max_val = np.max(col)

        if max_val > min_val:
            # D_norm[:, j] = (col - min_val) / (max_val - min_val)
            D_norm[:, j] = col/max_val
        else:
            D_norm[:, j] = 0.0

    return D_norm


In [5]:
def compute_frequency(D_norm):
    """
    Compute frequency-based weights from row-wise minima.

    Parameters
    ----------
    D_norm : ndarray, shape (N, L)

    Returns
    -------
    w : ndarray, shape (L,)
        Frequency weights
    """
    N, L = D_norm.shape

    min_indices = np.argmin(D_norm, axis=1)
    w = np.zeros(L)
    for j in range(L):
        w[j] = np.sum(min_indices == j) / N

    return min_indices


In [ ]:
def lagged_kernel_feature_score(
    x,
    y,
    L,
    gamma_x=1.0,
    gamma_y=1.0
):
    """
    Full algorithm pipeline.

    Returns
    -------
    score : float
    details : dict
        Intermediate matrices and weights
    """
    X_lag = create_lags(x, L)
    y_aligned = y[L:]

    D = kernel_distance_columnwise(
        X_lag, y_aligned, gamma_x, gamma_y
    )

    # D_norm = normalize_columns(D)
    D = np.nan_to_num(D, nan=0.0) # because of zero division in normalization using frobenius norm
    min_indices = compute_frequency(D)
    indice_freq = np.bincount(min_indices)
    indice_percent = indice_freq / len(min_indices)

    return {
        "D": D,
        # "D_norm": D_norm,
        "min_indices": min_indices,
        "indice_freq": indice_freq,
        "indice_percent": indice_percent,
    }


In [66]:
import numpy as np

np.random.seed(0)
N = 1000

# Generate depedent time series
x = np.array([1, 4, 3, 4, 4, 6, 6, 8, 8, 5, 3, 2, 4, 6, 7, 9, 8, 6, 5, 4, 3])
y = np.array([1, 0, 2, 0, 2, 1, 1, 2, 1, 0, 5.5, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0])


L = 2
result = lagged_kernel_feature_score(x, y, L, gamma_x=0.5, gamma_y=0.5)
print("Feature Relevance Score:", result["indice_percent"])

Feature Relevance Score: [0.84210526 0.15789474]


/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(


In [13]:
result["indice_freq"]

array([ 14,   8,   6,   5,   4,   4,   4,   3,   2,   3,   3,   3,   3,
         3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,
         3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,   3,
         3,   3,   3,   3,   3,   3,   3,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         2,   2,   2,   3,   3,   3,   3,   4, 627])

In [56]:
import numpy as np

np.random.seed(0)
N = 2000

# Independent feature and target
x = np.random.randn(N)  # independent standard normal
y = np.random.randn(N)  # independent standard normal

L = 40
result = lagged_kernel_feature_score(x, y, L, gamma_x=0.5, gamma_y=0.5)
print("Feature Relevance Score:", result["indice_percent"])

/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.

Feature Relevance Score: [0.03571429 0.02602041 0.02755102 0.02959184 0.0244898  0.02295918
 0.02908163 0.02346939 0.02295918 0.02704082 0.02602041 0.02397959
 0.025      0.02704082 0.02244898 0.02346939 0.02244898 0.02244898
 0.025      0.02397959 0.02244898 0.01887755 0.03112245 0.02244898
 0.02193878 0.02806122 0.02653061 0.02142857 0.02295918 0.02653061
 0.025      0.01989796 0.02091837 0.03061224 0.02295918 0.02653061
 0.02091837 0.02704082 0.0244898  0.02857143]


/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
/var/folders/94/18sqr3292qj66m2k622fh4580000gn/T/ipykernel_21464/1111320835.py:38: RuntimeWarning: invalid value encountered in sqrt
  D[:, j] = np.sqrt(
